# Microscope Acquisition with Pycro-manager

This notebook demonstrates how to control the microscope using the high-level `Acquisition` API and `multi_d_acquisition_events`.

## Requirements
1. Micro-Manager running with Pycro-manager server enabled.
2. `pycromanager` installed.

In [ ]:
from pycromanager import Bridge, Acquisition, multi_d_acquisition_events
import random
import time

In [ ]:
# Connect to the Micro-Manager Core via Bridge to get initial state
bridge = Bridge()
core = bridge.get_core()

xy_stage = core.get_xy_stage_device()
print(f"Using XY Stage: {xy_stage}")

if xy_stage:
    start_x = core.get_x_position(xy_stage)
    start_y = core.get_y_position(xy_stage)
    print(f"Initial Position: X={start_x}, Y={start_y}")
else:
    # Fallback for testing if no stage present
    start_x, start_y = 0, 0
    print("No XY Stage found. Using (0,0).")

In [ ]:
def generate_random_acquisition_events(num_rounds=5, moves_per_round=10, range_um=100):
    """
    Generate a list of acquisition events.
    Structure: 
    - For each round:
        - 10 events at random positions.
        - 1 event returning to start position (to satisfy the 'return to start' requirement).
    """
    events = []
    
    for r in range(num_rounds):
        # 1. Ten random moves
        for m in range(moves_per_round):
            # Calculate random target
            dx = random.uniform(-range_um, range_um)
            dy = random.uniform(-range_um, range_um)
            
            evt = {
                'axes': {'round': r, 'position': m, 'type': 'random_move'},
                'x': start_x + dx,
                'y': start_y + dy,
                'exposure': 1 # camera exposure time in ms
            }
            events.append(evt)
        
        # 2. Return to start (Imaging at start to confirm return)
        # We use a distinct axis or label to mark this.
        evt_return = {
            'axes': {'round': r, 'position': moves_per_round, 'type': 'return_home'},
            'x': start_x,
            'y': start_y,
        }
        events.append(evt_return)
        
    return events

events_list = generate_random_acquisition_events(num_rounds=5, moves_per_round=10)
print(f"Generated {len(events_list)} events.")
# Show first few events
print(events_list[:3])

In [ ]:
# Define a directory to save data (optional)
save_dir = r"C:\Users\Public\Documents\AcquisitionData"
acq_name = "RandomWalkAcquisition"

# Run the Acquisition
# The Acquisition class automatically handles the event loop
print("Starting Acquisition...")
with Acquisition(directory=save_dir, name=acq_name) as acq:
    acq.acquire(events_list)
    
print("Acquisition Complete.")